In [1]:
from ortools.sat.python import cp_model

In [2]:
p = 127

def restrict(model, expr, lb, ub):
    model.Add(lb <= expr)
    model.Add(expr <= ub)

def model_copy(model, u, v0, v1):
    e = model.NewBoolVar("")
    model.Add(e*(p-1) + u == v0 + v1)
    restrict(model, v0+v1-e*p, 0, p-1)

def model_square(model, u, v):
    e = model.NewBoolVar("")
    model.Add(e*(p-1) + u == 2*v)
    restrict(model, 2*v-e*p, 0, p-1)
    

def model_addition(model, u0, u1, v):
    model.Add(v == u0 + u1)

def model_constant_addition(model, u, v):
    model.Add(v >= u)

In [3]:
def model_F(model, u): # return v
    u = list(u)
    
    # constant addition
    v = model.NewIntVar(0, p-1, "")
    model_constant_addition(model, u[3], v)
    u[3] = v
    
    # squaring feistel
    for i in range(3):
        # extra vars
        v = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        # copy
        model_copy(model, u[i+1], v[0], v[1])
        u[i+1] = v[0]
        # square
        model_square(model, v[1], v[2])
        # add
        model_addition(model, u[i], v[2], v[3])
        u[i] = v[3]
    
    # linear layer (note we ignore constant multiplication)
    for r in range(4):
        for i in (0, 2):
            # extra vars
            v = [model.NewIntVar(0, p-1, "") for _ in range(3)]
            # copy
            model_copy(model, u[i], v[0], v[1])
            u[i] = v[0]
            # addition
            model_addition(model, u[i+1], v[1], v[2])
            u[i+1] = v[2]
        if r < 3: # shuffle
            u = u[-1:] + u[:-1]
    
     # constant addition
    v = model.NewIntVar(0, p-1, "")
    model_constant_addition(model, u[3], v)
    u[3] = v
    
    # squaring feistel again
    for i in range(3):
        # extra vars
        v = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        # copy
        model_copy(model, u[i+1], v[0], v[1])
        u[i+1] = v[0]
        # square
        model_square(model, v[1], v[2])
        # add
        model_addition(model, u[i], v[2], v[3])
        u[i] = v[3]
    
    # final shuffle
    return u[-1:] + u[:-1]

In [4]:
def model_round(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    # undo v rotation
    v = list(v)
    v = v[12:] + v[:12]
    # model feistel
    for i in (0, 8):
        # F
        w0 = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        w1 = model_F(model, w0)
        # copy and addition
        for j in range(4):
            # copy
            model_copy(model, u[i+j], w0[j], v[i+j])
            # addition
            model_addition(model, u[i+j+4], w1[j], v[i+j+4])

In [5]:
def model_step(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    exponents = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(4)] + [v]

    # key additions
    for i in range(16):
        model_constant_addition(model, u[i], exponents[0][i])

    # rounds
    for i in range(4):
        model_round(model, exponents[i], exponents[i+1])

# degree estimations

In [6]:
for r in range(1, 16):
    nsteps = r // 4
    extra_rounds = r % 4
    res = []
    for a in range(16):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == a:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        model.Maximize(sum(us[0]))
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL:
            res.append(int(solver.ObjectiveValue()))
            # print(a, f"maximal degree is: {solver.ObjectiveValue()}")
        elif status == cp_model.FEASIBLE:
            # print(a, f"lower bound on the degree is: {solver.ObjectiveValue()}")
            print(a, "error")
        else:
            print(a, "error")
    print(r, min(res), max(res), res)

1 1 4 [2, 4, 4, 4, 1, 1, 1, 1, 2, 4, 4, 4, 1, 1, 1, 1]
2 2 16 [8, 16, 16, 16, 2, 4, 4, 4, 8, 16, 16, 16, 2, 4, 4, 4]
3 8 64 [32, 64, 64, 64, 8, 16, 16, 16, 32, 64, 64, 64, 8, 16, 16, 16]
4 32 256 [128, 256, 256, 256, 32, 64, 64, 64, 128, 256, 256, 256, 32, 64, 64, 64]
5 128 571 [443, 571, 571, 571, 128, 256, 256, 256, 443, 571, 571, 571, 128, 256, 256, 256]
6 443 886 [758, 886, 886, 886, 443, 571, 571, 571, 758, 886, 886, 886, 443, 571, 571, 571]
7 758 1358 [1142, 1358, 1358, 1358, 758, 886, 886, 886, 1142, 1358, 1358, 1358, 758, 886, 886, 886]
8 1142 1781 [1610, 1781, 1781, 1781, 1142, 1358, 1358, 1358, 1610, 1781, 1781, 1781, 1142, 1358, 1358, 1358]
9 1610 1951 [1864, 1951, 1951, 1951, 1610, 1781, 1781, 1781, 1864, 1951, 1951, 1951, 1610, 1781, 1781, 1781]
10 1864 1999 [1974, 1999, 1999, 1999, 1864, 1951, 1951, 1951, 1974, 1999, 1999, 1999, 1864, 1951, 1951, 1951]
11 1974 2011 [2005, 2011, 2011, 2011, 1974, 1999, 1999, 1999, 2005, 2011, 2011, 2011, 1974, 1999, 1999, 1999]
12 2005 201

# finding max-data properties
i.e when are all component functions dense.

In [7]:
# check how far we can get
nsteps = 3
extra_rounds = 3
for a in range(16):
    for b in range(16):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 0 zero sum
0 1 zero sum
0 2 zero sum
0 3 zero sum
0 4 zero sum
0 5 zero sum
0 6 zero sum
0 7 zero sum
0 8 zero sum
0 12 zero sum
0 13 zero sum
0 14 zero sum
0 15 zero sum
1 4 zero sum
1 5 zero sum
1 6 zero sum
1 7 zero sum
1 8 zero sum
1 12 zero sum
2 4 zero sum
2 5 zero sum
2 6 zero sum
2 7 zero sum
2 8 zero sum
2 12 zero sum
3 4 zero sum
3 5 zero sum
3 6 zero sum
3 7 zero sum
3 8 zero sum
3 12 zero sum
4 4 zero sum
4 5 zero sum
4 6 zero sum
4 7 zero sum
4 12 zero sum
5 12 zero sum
6 12 zero sum
7 12 zero sum
8 0 zero sum
8 4 zero sum
8 5 zero sum
8 6 zero sum
8 7 zero sum
8 8 zero sum
8 9 zero sum
8 10 zero sum
8 11 zero sum
8 12 zero sum
8 13 zero sum
8 14 zero sum
8 15 zero sum
9 0 zero sum
9 4 zero sum
9 12 zero sum
9 13 zero sum
9 14 zero sum
9 15 zero sum
10 0 zero sum
10 4 zero sum
10 12 zero sum
10 13 zero sum
10 14 zero sum
10 15 zero sum
11 0 zero sum
11 4 zero sum
11 12 zero sum
11 13 zero sum
11 14 zero sum
11 15 zero sum
12 4 zero sum
12 12 zero sum
12 13 zero sum
12 14

In [8]:
# check how far we can get
nsteps = 4
extra_rounds = 0
for a in range(16):
    for b in range(16):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 4 zero sum
0 12 zero sum
0 13 zero sum
0 14 zero sum
0 15 zero sum
1 4 zero sum
2 4 zero sum
3 4 zero sum
8 4 zero sum
8 5 zero sum
8 6 zero sum
8 7 zero sum
8 12 zero sum
9 12 zero sum
10 12 zero sum
11 12 zero sum


In [9]:
# check how far we can get
nsteps = 4
extra_rounds = 1
for a in range(16):
    for b in range(16):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

# finding minimal data properties
On 13 rounds, because only max-data properties exist for 14 rounds

In [10]:
# check positions
nsteps = 3
extra_rounds = 1
eligible_positions = set()
for a in range(16):
    for b in range(12, 13):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                e = model.NewIntVar(1, 2, "")
                model.Add(us[0][i] == ((p-1)//2)*e) # subgroup of squares
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
            eligible_positions.add((a,b))
        else:
            print(a, b, "error")

0 12 zero sum
1 12 zero sum
2 12 zero sum
3 12 zero sum
8 12 zero sum


In [11]:
# we focus on output component 12
b = 12
pos = set(x[0] for x in eligible_positions if x[1] == b)
# for each position, check how low the data can go
mindegs = [0, 1, 2, 3, 6, 7, 9, 14, 18, 21, 42, 63][::-1] # from high to low
b = 12
eligible_degs = set()
for a in pos:
    for d in mindegs:
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                model.Add(us[0][i] >= d) # subgroup of squares
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            break
        elif status == cp_model.INFEASIBLE:
            print(a, d, "zero sum")
            eligible_degs.add((a, d))
        else:
            print(a, b, "error")
# so each position can be reduced to 0 data

0 63 zero sum
0 42 zero sum
0 21 zero sum
0 18 zero sum
0 14 zero sum
0 9 zero sum
0 7 zero sum
0 6 zero sum
0 3 zero sum
0 2 zero sum
0 1 zero sum
0 0 zero sum
1 63 zero sum
1 42 zero sum
2 63 zero sum
2 42 zero sum
3 63 zero sum
3 42 zero sum
8 63 zero sum
8 42 zero sum


In [12]:
from itertools import combinations
for x in combinations(eligible_degs, 2):
    a = {y[0]:y[1] for y in x}
    if len(a) != 2:
        continue
    model = cp_model.CpModel()
    us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
    
    # input constraints
    for i in range(16):
        if i in a:
            model.Add(us[0][i] >= a[i]) # subgroup of squares
        else:
            model.Add(us[0][i] == p-1)
        
    # model function
    for i in range(nsteps):
        model_step(model, us[i], us[i+1])
    for i in range(16):
        model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
    for i in range(extra_rounds):
        model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
    
    # output constraints
    for i in range(16):
        if i == b:
            model.Add(us[-1][i] == 1)
        else:
            model.Add(us[-1][i] == 0)
    
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 8
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        pass
    elif status == cp_model.INFEASIBLE:
        print(a, "zero sum")
    else:
        print(a, "error")
        break

In conclusion, minimal data is $127^{15} \approx 2^{104.83}$

# higher divisibility

In [13]:
p = 127

def model_addition_divisibility(model, u0, u1, v):
    e = model.NewBoolVar("")
    model.Add(v == u0 + u1 + e*(1-p))
    return e

def model_constant_addition_divisibility(model, u, v):
    k = model.NewIntVar(0, p-1, "")
    return model_addition_divisibility(model, u, k, v)

In [14]:
def model_F_divisibility(model, u): # return v
    u = list(u)
    cor = 0
    # constant addition
    v = model.NewIntVar(0, p-1, "")
    cor += model_constant_addition_divisibility(model, u[3], v)
    u[3] = v
    
    # squaring feistel
    for i in range(3):
        # extra vars
        v = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        # copy
        model_copy(model, u[i+1], v[0], v[1])
        u[i+1] = v[0]
        # square
        model_square(model, v[1], v[2])
        # add
        cor += model_addition_divisibility(model, u[i], v[2], v[3])
        u[i] = v[3]
    
    # linear layer (note we ignore constant multiplication)
    for r in range(4):
        for i in (0, 2):
            # extra vars
            v = [model.NewIntVar(0, p-1, "") for _ in range(3)]
            # copy
            model_copy(model, u[i], v[0], v[1])
            u[i] = v[0]
            # addition
            cor += model_addition_divisibility(model, u[i+1], v[1], v[2])
            u[i+1] = v[2]
        if r < 3: # shuffle
            u = u[-1:] + u[:-1]
    
     # constant addition
    v = model.NewIntVar(0, p-1, "")
    cor += model_constant_addition_divisibility(model, u[3], v)
    u[3] = v
    
    # squaring feistel again
    for i in range(3):
        # extra vars
        v = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        # copy
        model_copy(model, u[i+1], v[0], v[1])
        u[i+1] = v[0]
        # square
        model_square(model, v[1], v[2])
        # add
        cor += model_addition_divisibility(model, u[i], v[2], v[3])
        u[i] = v[3]
    
    # final shuffle
    return u[-1:] + u[:-1], cor

In [15]:
def model_round_divisibility(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    # undo v rotation
    v = list(v)
    v = v[12:] + v[:12]
    cor = 0
    # model feistel
    for i in (0, 8):
        # F
        w0 = [model.NewIntVar(0, p-1, "") for _ in range(4)]
        w1, ec = model_F_divisibility(model, w0)
        cor += ec
        # copy and addition
        for j in range(4):
            # copy
            model_copy(model, u[i+j], w0[j], v[i+j])
            # addition
            cor += model_addition_divisibility(model, u[i+j+4], w1[j], v[i+j+4])
    return cor

In [16]:
def model_step_divisibility(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    exponents = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(4)] + [v]
    cor = 0
    # key additions
    for i in range(16):
        cor += model_constant_addition_divisibility(model, u[i], exponents[0][i])

    # rounds
    for i in range(4):
        cor += model_round_divisibility(model, exponents[i], exponents[i+1])
    return cor

In [17]:
# check positions
nsteps = 1
extra_rounds = 2
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^3.0


In [18]:
# check positions
nsteps = 1
extra_rounds = 3
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^2.0


In [19]:
# check positions
nsteps = 2
extra_rounds = 0
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^2.0


In [20]:
# check positions
nsteps = 2
extra_rounds = 1
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^2.0


In [21]:
# check positions
nsteps = 2
extra_rounds = 2
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^2.0


In [22]:
# check positions
nsteps = 2
extra_rounds = 3
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^2.0


In [23]:
# check positions
nsteps = 3
extra_rounds = 0
a = 0
b = 12
model = cp_model.CpModel()
us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
cor = 0
# input constraints
for i in range(16):
    if i == a:
        model.Add(us[0][i] == p-2)
    else:
        model.Add(us[0][i] == p-1)
    
# model function
for i in range(nsteps):
    cor += model_step_divisibility(model, us[i], us[i+1])
for i in range(16):
    cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
for i in range(extra_rounds):
    cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])

# output constraints
for i in range(16):
    if i == b:
        model.Add(us[-1][i] == 1)
    else:
        model.Add(us[-1][i] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 8
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print(a, b, "zero sum")
else:
    print(a, b, "error")

divisibility by p^1.0
